# Install Required Software with Conda

**Create the Conda Software Environment for the opesarlab_OPERA-RTC-S1_Recipe_Book**


<div class="alert alert-info" style="display: flex; align-items: center; font-family: 'Times New Roman', Times, serif; background-color: #d1ecf1;">
  <div style="display: flex; align-items: center; width: 10%;">
    <a href="https://github.com/ASFOpenSARlab/opensarlab_OPERA-CSLC_Recipe_Book/issues">
      <img src="https://opensarlab-docs.asf.alaska.edu/opensarlab-notebook-assets/logos/github_issues.png" alt="GitHub logo over the word Issues" style="width: 100%;">
    </a>
  </div>
  <div style="width: 95%;">
    <b>Did you find a bug? Do you have a feature request?</b>
    <br/>
    Explore GitHub Issues on this Jupyter Book's GitHub repository. Find solutions, add to the discussion, or start a new bug report or feature request: <a href="https://github.com/ASFOpenSARlab/opensarlab_OPERA-RTC-S1_Recipe_Book/issues">opensarlab_OPERA-RTC-S1_Recipe_Book Issues</a>
  </div>
</div>

<div class="alert alert-info" style="display: flex; align-items: center; justify-content: space-between; font-family: 'Times New Roman', Times, serif; background-color: #d1ecf1;">
  <div style="display: flex; align-items: center; width: 10%; margin-right: 10px;">
    <a href="mailto:uso@asf.alaska.edu">
      <img src="https://opensarlab-docs.asf.alaska.edu/opensarlab-notebook-assets/logos/ASF_support_logo.png" alt="ASF logo" style="width: 100%">
    </a>
  </div>
  <div style="width: 95%;">
    <b>Have a question related to SAR, OPERA RTCs, or ASF data access?</b>
    <br/>
    Contact ASF User Support: <a href="mailto:uso@asf.alaska.edu">uso@asf.alaska.edu</a>
  </div>
</div>

---
## 0. Set the Correct Conda Enviroment Prefix if Working in OpenSARLab

OpenSARLab requires a specific environment prefix path to create a persistent environment that Jupyter will recognize.


If you are working in [OpenSARLab](https://asf.alaska.edu/asf-services-open-science-lab/), we will use `~/.local/envs`, otherwise we will use your system's conda `root_prefix` path.

In [ ]:
import ipywidgets as widgets

osl_option = widgets.RadioButtons(
    options=['In OpenSARLab', 'Not in OpenSARLab'],
    description='Where are you working?',
    disabled=False,
    layout=widgets.Layout(min_width='800px')
)
display(osl_option)

In [ ]:
from pathlib import Path

env_config_dir = Path.cwd() / 'environments'

in_osl = 'In' in osl_option.value

if in_osl:
    conda_prefix = Path.home() / '.local/envs'
else:
    conda_info = subprocess.run(
        ['conda', 'run', '-n', 'base', 'conda', 'info', '--json'], 
        capture_output=True, 
        text=True
    )
    info = json.loads(conda_info.stdout)
    conda_prefix = Path(f"{info['root_prefix']}/envs")
print(f'conda prefix: {conda_prefix}')

---
## 1. Build the `osl_opera_cslc_jbook_env` Conda Environment

**Select whether to create the environment from configs with explicitly pinned package versions or from unpinned configs**

After running the following code cell:
- Select `Pinned` for maximum (but not guaranteed) stability when creating the environment on a Linux x86 system
- Select `Unpinned` to attempt installing the latest package versions or if on a non-Linux x86 system

In [ ]:
config_options = widgets.RadioButtons(
    options=['Pinned (recommended)', 'Unpinned'],
    description='Create the environment from pinned or unpinned configs',
    disabled=False,
    layout=widgets.Layout(min_width='800px')
)
display(config_options)

In [ ]:
import yaml

pinned = 'Pinned' in config_options.value
if pinned:
    yaml_name =  'opensarlab_OPERA-RTC-S1_recipe_book_locked_env.yaml'
else:
    yaml_name = 'opensarlab_OPERA-RTC-S1_recipe_book_unlocked_env.yaml'

conda_yaml_pth = list(env_config_dir.glob(yaml_name))[0]
with open(conda_yaml_pth, 'r') as f:
    env = yaml.safe_load(f)
    env_name = env['name']

env_prefix = conda_prefix / env_name

print(f'environment name: {env_name}')
print(f'environment prefix: {env_prefix}')

In [ ]:
!mamba clean -p -t --yes
!pip cache purge

In [ ]:
import shutil

# Create the environment
!mamba env create -f $conda_yaml_pth --prefix $env_prefix --force

# Provide access to the environment in notebooks as a Jupyter kernel named for the environment 
!{env_prefix}/bin/python -m ipykernel install --prefix {env_prefix} --name {env_name}

# Having created a well-named kernel, and in case kernda is installed in base,
# delete the default kernel named 'Python' to avoid duplicate buttons in the Launcher
shutil.rmtree(env_prefix/'share/jupyter/kernels/python3')